# Searching rotation center

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os, glob
import nibabel as nib

from scipy.ndimage.interpolation import rotate
from scipy.ndimage.interpolation import shift

from ipywidgets import interact
import ipywidgets as widgets

In [2]:
ROOT = "D://Data/"

pat_list = os.listdir(ROOT)
pat_path = [os.path.join(ROOT, pat) for pat in pat_list]

ct_list = [glob.glob(path+'/r*.nii') for path in pat_path]

np_ct = []
for i in ct_list:
    if i != [] :
        img = nib.load(i[0])
        np_ct.append(np.array(img.dataobj))

ct = np.array(np_ct)
np.save('C:\\Users\\Heeguen\\Desktop\\heeguen\\Dental_panorama_2020\\np_data\\register_ct.npy', ct)

In [2]:
ct = np.load('C:\\Users\\Heeguen\\Desktop\\heeguen\\Dental_panorama_2020\\np_data\\register_ct.npy')
trans_ct = np.transpose(ct, (0, 3, 1, 2))
print(trans_ct.shape)

(6, 424, 671, 671)


In [3]:
def show(x, y, z):
    plt.figure(figsize=(24,12))
    plt.subplot(121)
    plt.imshow(trans_ct[x, y, ...], cmap='gray')
    
    plt.subplot(122)
    plt.imshow(np.rot90(ct[x, z, ...]), cmap='gray')    
    
interact(show, x=widgets.IntSlider(min=0, max=(trans_ct.shape[0]-1), step=1,value=0), 
         y=widgets.IntSlider(min=0, max=(trans_ct.shape[1]-1), step=1,value=100), 
         z=widgets.IntSlider(min=0, max=(ct.shape[1]-1), step=1,value=300))

interactive(children=(IntSlider(value=0, description='x', max=5), IntSlider(value=100, description='y', max=42…

<function __main__.show(x, y, z)>

In [29]:
def show(x, y, z):
    plt.figure(figsize=(24,12))
    plt.subplot(121)
    plt.imshow(trans_ct[x, y, ...], cmap='gray')
    
    plt.subplot(122)
    plt.hist(trans_ct[x, y, ...])    
    
interact(show, x=widgets.IntSlider(min=0, max=(trans_ct.shape[0]-1), step=1,value=0), 
         y=widgets.IntSlider(min=0, max=(trans_ct.shape[1]-1), step=1,value=0), 
         z=widgets.IntSlider(min=0, max=(trans_ct.shape[1]-1), step=1,value=0))

interactive(children=(IntSlider(value=0, description='x', max=5), IntSlider(value=0, description='y', max=423)…

<function __main__.show(x, y, z)>

In [4]:
trans_ct.shape

(6, 424, 671, 671)

</br>

## 위의 image 처럼 ct의 coregistration 과정에 의해, ct 중 잘린 부분이 생김. 

## 그럼 부분들을 보상해주기 위해 해당 부분을 영상의 minimum 값으로 바꾸어줌. 

In [66]:
sam = trans_ct[:, 20:200, ...]

for p in range(sam.shape[0]):
    for s in range(sam.shape[1]):
        for w in range(sam.shape[2]):
            if w < 50 :
                for h in range(sam.shape[3]):
                    if sam[p,s,w,h]==0 :
                        sam[p,s,w,h] = sam_min

            elif w >= 50:
                for h in range(sam.shape[3]):
                    if h > 580 and sam[p,s,w,h]==0 :
                        sam[p,s,w,h] = sam_min

            elif w >= 650:
                for h in range(sam.shape[3]):
                    if sam[p,s,w,h]==0 :
                        sam[p,s,w,h] = sam_min

print('after processing : ', sam.shape)

after processing :  (6, 180, 671, 671)


In [ ]:
(6,180, 641, 600)

In [67]:
def show1(x, y):
    plt.figure(figsize=(12,12))
#     plt.subplot(121)
    plt.imshow(np.rot90(sam[x, y, ...]), cmap='gray')
    
#     plt.subplot(122)
#     plt.imshow()

interact(show1, x=widgets.IntSlider(min=0, max=(sam.shape[0]-1), step=1,value=0),
        y=widgets.IntSlider(min=0, max=(sam.shape[1]-1), step=1,value=0))

interactive(children=(IntSlider(value=0, description='x', max=5), IntSlider(value=0, description='y', max=179)…

<function __main__.show1(x, y)>

</br>

# 보상해주기 위해 해당 부분을 영상의 minimum 값으로 바꾸어주는 방법보다 slicing 하는 방법이 더 좋아 보임.

In [5]:
sam = trans_ct[:, 20:200, 30:, :600]

print('after processing : ', sam.shape)

after processing :  (6, 180, 641, 600)


In [11]:
def show(x, y, z):
    plt.figure(figsize=(24,12))
    plt.subplot(121)
    plt.imshow(sam[x, y, ...], cmap='gray')
    
    plt.subplot(122)
    plt.imshow(np.rot90(sam[x, z, ...]), cmap='gray')    
    
interact(show, x=widgets.IntSlider(min=0, max=(sam.shape[0]-1), step=1,value=0), 
         y=widgets.IntSlider(min=0, max=(sam.shape[1]-1), step=1,value=0), 
         z=widgets.IntSlider(min=0, max=(sam.shape[1]-1), step=1,value=0))

interactive(children=(IntSlider(value=0, description='x', max=5), IntSlider(value=0, description='y', max=179)…

<function __main__.show(x, y, z)>

</br>

## summation

In [6]:
ct_sum = np.zeros((sam[0].shape))

for s in range(sam.shape[0]):
    ct_sum += sam[s]
    
ct_sum.shape

(180, 641, 600)

In [7]:
def show1(x):
    plt.figure(figsize=(12,12))
#     plt.subplot(121)
    plt.imshow(np.rot90(ct_sum[x]), cmap='gray')
    
#     plt.subplot(122)
#     plt.imshow()

interact(show1, x=widgets.IntSlider(min=0, max=(ct_sum.shape[0]-1), step=1,value=0))

interactive(children=(IntSlider(value=0, description='x', max=179), Output()), _dom_classes=('widget-interact'…

<function __main__.show1(x)>